In [5]:
import pandas as pd
import pickle
import re
#Se importa el archivo pickle
with open('scrap_info_alq.pkl', 'rb') as f:
    diccionario = pickle.load(f)


with open('scrap_info.pkl', 'rb') as f:
    diccionario2 = pickle.load(f)
#Se transforma el diccionario en un dataframe
df = pd.DataFrame(diccionario)
df2 = pd.DataFrame(diccionario2)
#Ahora se cambia las filas por las columnas
df = df.T
df2 = df2.T
df.head()
df2.head()

#Ahora unimos los dos dataframes
df_def = pd.concat([df,df2])

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re

# Descargar los recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    if text is None:
        return ''
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)
    # Tokenización
    tokens = nltk.word_tokenize(text)
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

descripcion_filtrada = df_def['Descripcion'].apply(preprocess_text)

print(descripcion_filtrada)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


https://www.indomio.es/anuncios/95227781/    alquila piso amueblado habitaciones carrer del...
https://www.indomio.es/anuncios/95146207/    piso estudiantes entrar partir ro febrero juni...
https://www.indomio.es/anuncios/94613177/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/94500757/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/95223135/    rincón acogedor necesito espacio gustará aquí ...
                                                                   ...                        
https://www.indomio.es/anuncios/95007509/    atico m valencia valencia solidinmuebles vende...
https://www.indomio.es/anuncios/94958481/    venden pisos obra nueva camino moncada trata r...
https://www.indomio.es/anuncios/95006805/    vende exclusiva viviendas obra nueva calle arq...
https://www.indomio.es/anuncios/95313313/    vende piso barrio tormoseste luminoso apartame...
https://www.indomio.es/anuncios/95319871/    piso 

## LSA

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

tv = TfidfVectorizer()
svd = TruncatedSVD(n_components=100)
lsa = make_pipeline(tv, svd, Normalizer(copy=False))
lsa_topic_vectors = lsa.fit_transform(descripcion_filtrada)

lsa_topic_vectors

array([[ 0.14497312, -0.26972876,  0.78737534, ...,  0.02087234,
        -0.10260844,  0.06362454],
       [ 0.18997783, -0.19594662,  0.03322387, ...,  0.05826937,
        -0.0366585 ,  0.09769377],
       [ 0.28900457, -0.31092992, -0.08249069, ...,  0.04435693,
         0.11768029,  0.0356905 ],
       ...,
       [ 0.25781214, -0.23456174, -0.06241935, ..., -0.0810347 ,
         0.02747867, -0.05118084],
       [ 0.39467638, -0.38163988,  0.00312988, ...,  0.06312087,
        -0.18018953,  0.16932089],
       [ 0.31317552, -0.33143523,  0.01432733, ..., -0.00864162,
         0.03434649, -0.03294247]])

In [8]:
# Nueva descripción
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción usando la misma pipeline LSA
vector_descripcion = lsa.transform([descripcion])

# Calcular la similitud coseno entre la nueva descripción y todas las descripciones existentes
similarities = cosine_similarity(vector_descripcion, lsa_topic_vectors)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Opcionalmente, puedes mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()


Descripción: DISPONIBLE A PARTIR DE JULIO 2024. ALQUILER PARA ESTUDIANTES UNIVERSITARIOS (CONTRATO DE TEMPORADA). Se alquila este precioso apartamento , semireformado, en un edificio tranquilo y en una zona ideal si te gusta tener todo a un paso. Se trata de una vivienda con un total de 4 habitaciones y dos cuartos de baños completos. el apartamento se distribuye en zona de salón comedor con acceso a balcón y aire acondicionado (frío /calor), 4 habitaciones con camas de 1,35, totalmente equipadas y dos cuartos de baños completos. Cocina independiente, con todo lo que puedas necesitar en tu día a día y con acceso directo a terraza en zona común, pero de uso privativo. Como ves el piso lo tiene todo, es bonito, esta cerca de transporte público, tanto metro como autobús en menos de 9 minutos a pie, 5 minutos andando de supermercado y el mercado municipal del Cabañal a 7 minutos y lo mejor de todo, a tan solo 16 minutos a pie de la playa. No se puede pedir más. Si estás buscando una vivien

## LDA

In [9]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)

tf = tf_vectorizer.fit_transform(descripcion_filtrada)

lda = LatentDirichletAllocation(n_components=10, max_iter=5,
learning_method='online',
learning_offset=50.,
random_state=0)

lda_topic_matrix = lda.fit_transform(tf)

In [10]:
# Nueva descripción
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción
vector_descripcion = tf_vectorizer.transform([descripcion])
vector_descripcion_lda = lda.transform(vector_descripcion)

# Calcular la similitud coseno entre la nueva descripción y las descripciones existentes
similarities = cosine_similarity(vector_descripcion_lda, lda_topic_matrix)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()

Descripción: Bonito piso recién reformado. Está todo a estrenar! A tan solo unos minutos de la plaza del Ayuntamienti y del Mercado Central. 
Similitud: 0.9999964260327222

Descripción: Piso en pleno centro de Valencia de 545 m2 con terrazas muy grandes. Dispone de un apartamento añadido para los invitados.el piso es de estilo señorial con acabados de primera calidad reformado y a estrenar.
Similitud: 0.9999261534660971

Descripción: Apartamento de 3 dormitorios en una excelente ubicación, Restaurantes, supermercados cercas.El dormitorio principal tiene un vestidor y un baño en suite. La cocina está amueblada con área de lavandería separada. Aire acondicionado
Similitud: 0.9999139048645553

Descripción: Un encantador apartamento con lindas paredes azules en la sala de estar, una espaciosa cocina y una gran terraza en un vecindario tranquilo y residencial. El apartamento está a solo 10 minutos a pie del puerto de Valencia, y cerca encontrará algunas tiendas, una piscina municipal y una 

### Comprobar la coherencia de los topics LDA

In [11]:
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

# Paso 1: Preparar datos para gensim
# Convertir el texto tokenizado (asumiendo que tienes 'descripcion_filtrada' como una lista de listas de palabras)
texts = [doc.split() for doc in descripcion_filtrada]

# Crear un diccionario de Gensim a partir de los textos
dictionary = Dictionary(texts)

# Crear un corpus de Gensim utilizando el diccionario
corpus = [dictionary.doc2bow(text) for text in texts]

# Paso 2: Extraer los tópicos del modelo LDA de scikit-learn
topics = []
feature_names = tf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    topics.append([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]])

# Paso 3: Calcular la coherencia del modelo
cm = CoherenceModel(model=None, topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("Coherencia del modelo LDA:", coherence)


ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\pjcan\anaconda3\envs\prueba\lib\site-packages\scipy\linalg\__init__.py)

## NMF

In [ ]:
# Inicialización de CountVectorizer
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
tf = tf_vectorizer.fit_transform(descripcion_filtrada)

# Uso de NMF
nmf = NMF(n_components=10, random_state=0, max_iter=300)

# Transformar los datos con NMF
nmf_topic_matrix = nmf.fit_transform(tf)

: 

In [ ]:
# Nueva descripción a comparar
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción usando el mismo vectorizador
vector_descripcion = tf_vectorizer.transform([descripcion])
vector_descripcion_nmf = nmf.transform(vector_descripcion)

# Calcular la similitud coseno entre la nueva descripción y las descripciones existentes transformadas
similarities = cosine_similarity(vector_descripcion_nmf, nmf_topic_matrix)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()

: 

### Comprobar la coherencia de los topics NMF

In [ ]:
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

# Paso 1: Preparar datos para gensim
texts = [doc.split() for doc in descripcion_filtrada]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Paso 2: Extraer los tópicos del modelo NMF
topics = []
feature_names = tf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf.components_):
    top_features_ind = topic.argsort()[:-10 - 1:-1]  # obtener los índices de las palabras más importantes
    topic_words = [feature_names[i] for i in top_features_ind]
    topics.append(topic_words)

# Paso 3: Calcular la coherencia del modelo
cm = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("Coherencia del modelo NMF:", coherence)


: 

### Mostrar palabras clave de cada tema en NMF

In [ ]:
# Obtener las palabras del vocabulario
feature_names = tf_vectorizer.get_feature_names_out()

# Función para mostrar las palabras clave por tema
def show_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx + 1}:")
        # Muestra las n_top_words palabras más importantes por tema
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        top_features_str = ", ".join(top_features)
        print(f"Palabras clave: {top_features_str}\n")

# Mostrar las palabras clave de cada tema
n_top_words = 10
show_top_words(nmf, feature_names, n_top_words)

: 

## HDI

In [ ]:
import pandas as pd
from gensim import similarities
from gensim.corpora import Dictionary
from gensim.models import HdpModel
from gensim.matutils import sparse2full

# Crear un diccionario de características y filtrar palabras extremadamente frecuentes o infrecuentes
texts = [doc.split() for doc in descripcion_filtrada]
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.95)

# Crear el corpus BOW
corpus = [dictionary.doc2bow(text) for text in descripcion_filtrada]

# Entrenar el modelo HDI
hdi = HdpModel(corpus, id2word=dictionary)

# Función para obtener la distribución de tópicos de un documento
def get_topic_distribution(text):
    bow = dictionary.doc2bow(text.lower().split())
    return sparse2full(hdi[bow], len(hdi.id2word))

# Convertir todos los documentos a vectores de tópicos
topic_vectors = [get_topic_distribution(text) for text in descripcion_filtrada]

: 

In [ ]:
# Nueva descripción
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"
descripcion_vec = get_topic_distribution(descripcion)

# Calcular la similitud coseno entre la nueva descripción y todas las descripciones existentes
index = similarities.MatrixSimilarity([descripcion_vec] + topic_vectors)
sims = index[descripcion_vec]

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = sims.argsort()[::-1]

# Opcionalmente, muestra las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {sims[index]}")
    print()

: 